In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

In [2]:
table1= (
    spark.read.csv(
        "../dataset/New/credit_score_detail_3w_report.csv", 
        header=True,
        inferSchema=True
    )
    .select(
        "FAC NO",
        "Branch Name",
        "PRODUCT Name",
        "Equipment Category Description",
        "Scheme Description",
        "Make Description",
        "District Name",
        "Connex Date",
        "Lease Amount",
        "Portfolio",
        "Int Rate",
        "Future Capital",
        "TOTAL DUE",
        "TOTAL COLLECTION",
        "TOTAL COLLECTION RATIO",
        "Collection For The Month",
        "Collection Ratio",
        "DPD",
        "90>=day_contract",
        "Credit Decision",
        "Credit Score"
    )
    .withColumn("good", f.when(f.col("90>=day_contract") == 0, 1).otherwise(0))
    .withColumn("bad", f.when(f.col("90>=day_contract") == 1, 1).otherwise(0))
#     .withColumn("good/bad(total_collection_ratio)",f.when(f.col("Collection Ratio")>=0.90,1).otherwise(0))
#     .withColumn("good_total_collection", f.when(f.col("good/bad(total_collection_ratio") == 1, 1).otherwise(0))
)
table2=(
    spark.read.csv(
        "../dataset/New/3wdata.csv", 
        header=True,
        inferSchema=True
    )
    
)

In [3]:
table2.describe()

DataFrame[summary: string, FAC NO: string, Product: string, Product Over All score: string, CUST_PERMRES: string, CUST PERMRES SCORE: string, Guarantor PreNises: string, Guarantor PreNises Over All score: string, Exposure: string, Exposure Category(Score): string, DSCR: string, DCSR over all score: string, Final Score: string]

In [4]:
master_table=(
    table1
    .join(table2,"FAC NO","left")
    .withColumn("CUST_PERMRES",f.when(f.col("CUST_PERMRES")=="0","Rent").otherwise(f.col("CUST_PERMRES")))
    .withColumn("Guarantor PreNises",f.when(f.col("Guarantor PreNises")=="0","No Guarantors").otherwise(f.col("Guarantor PreNises")))
    .withColumn("Guarantor PreNises",f.when(f.col("Guarantor PreNises")=="With Parents/Spouse","With Parents/spouse").otherwise(f.col("Guarantor PreNises")))
)

In [5]:
m=(table1
    .join(table2,"FAC NO","inner"))

In [6]:
m.count()

15096

In [7]:
master_table.show()

+-------------------+--------------------+--------------------+------------------------------+--------------------+----------------+-------------+-----------+------------+---------+--------+--------------+---------+----------------+----------------------+------------------------+----------------+----+----------------+---------------+------------+----+---+-------------+----------------------+------------------+------------------+------------------+---------------------------------+--------+------------------------+---------+-------------------+-----------+
|             FAC NO|         Branch Name|        PRODUCT Name|Equipment Category Description|  Scheme Description|Make Description|District Name|Connex Date|Lease Amount|Portfolio|Int Rate|Future Capital|TOTAL DUE|TOTAL COLLECTION|TOTAL COLLECTION RATIO|Collection For The Month|Collection Ratio| DPD|90>=day_contract|Credit Decision|Credit Score|good|bad|      Product|Product Over All score|      CUST_PERMRES|CUST PERMRES SCORE|Guaran

In [8]:
master_table_pd=master_table.toPandas()

In [9]:
master_table_pd.head()

,FAC NO,Branch Name,PRODUCT Name,Equipment Category Description,Scheme Description,Make Description,District Name,Connex Date,Lease Amount,Portfolio,...,Product Over All score,CUST_PERMRES,CUST PERMRES SCORE,Guarantor PreNises,Guarantor PreNises Over All score,Exposure,Exposure Category(Score),DSCR,DCSR over all score,Final Score
0,'002300091997050201,EMBILIPITIYA,Three Wheeler-Lease-Registered,Three Wheeler,3W-LE-Registered - Individual,BAJAJ,RATNAPURA,2012-05-21,"180,000",0,...,None,None,NaN,None,NaN,None,None,None,None,None
1,'007000286220050801,MAHIYANGANAYA,CASH IN HAND,Three Wheeler,3W Cash In Hand - ZERO PP,BAJAJ,KANDY,2020-12-23,"393,774",346218,...,None,None,NaN,None,NaN,None,None,None,None,None
2,'014200183766050801,VIRTUAL_OPERATION_POLGAHAWELA,CASH IN HAND,Three Wheeler,3W Cash In Hand - ZERO PP,BAJAJ,KURUNEGALA,2022-06-29,"177,390",143315,...,0.15,With Billing Proof,0.2,No Guarantors,0.00,28.28,0.25,3.2785542,0.05,0.870275
3,'001200772724050203,WELLAWATTA,Three Wheeler-Lease-Registered,Three Wheeler,3W-LE-Registered - Individ-ZERO PP,BAJAJ,NUWARA ELIYA,2021-11-01,"573,013",553747,...,None,None,NaN,None,NaN,None,None,None,None,None
4,'041200813926050803,EHELIYAGODA_UNIT_A,CASH IN HAND,Three Wheeler,3W Cash In Hand - ZERO PP,BAJAJ,COLOMBO,2022-05-23,"338,698",332284,...,0.15,Rent,0.0,Rent,0.02,28.17,0.25,5.3452116,0.05,0.57125


In [55]:
master_table_pd.to_csv("../output/final_output/master_table.csv")

## Analysis

In [11]:
master_table_new= (
    master_table
    .withColumn("CUST_PERMRES",f.when(f.col("CUST_PERMRES")=="With Parents/Spouse","With Parents/spouse").otherwise(f.col("CUST_PERMRES")))
    .filter(f.col("Connex Date").between("2022-04-01","2023-04-01"))
#     .filter(~f.col("Product").isNull())
#     .filter(~f.col("CUST_PERMRES").isNull())
#     .filter(~f.col("Guarantor PreNises").isNull())
#     .filter(~f.col("Exposure").isNull())
#     .filter(f.col("Exposure")<100)
)

In [12]:
master_table_new.count()

7676

In [13]:
master_table_new_collection_ratio=(
    master_table_new.groupBy("Product","90>=day_contract")
    .agg(f.count("FAC NO"),
         f.sum("TOTAL COLLECTION").alias("Total_collection"),
         f.sum("TOTAL DUE").alias("Total_due")
        )
    .withColumn("Total_Collection_Ratio",f.col("Total_collection")/f.col("Total_due"))
    .withColumn("90>=day_contract",f.when(f.col("90>=day_contract")==0,"good_customer").otherwise("bad_customer"))
    
)

In [14]:
master_table_new_collection_ratio.show()

+-------------+----------------+-------------+----------------+----------+----------------------+
|      Product|90>=day_contract|count(FAC NO)|Total_collection| Total_due|Total_Collection_Ratio|
+-------------+----------------+-------------+----------------+----------+----------------------+
|Registered 3W|   good_customer|         3381|      1083135181|1158051717|    0.9353081257941781|
| Cash in Hand|   good_customer|         2902|       891400202| 948068278|    0.9402278535048717|
|         #N/A|   good_customer|           13|         3942879|   4400234|    0.8960612094720417|
|Registered 3W|    bad_customer|          930|       188380269| 335384409|     0.561684633944925|
| Cash in Hand|    bad_customer|          446|        98339564| 159701929|    0.6157694187901763|
|         #N/A|    bad_customer|            4|          386214|   1019860|   0.37869315396230857|
+-------------+----------------+-------------+----------------+----------+----------------------+



# Taking good/bad as target variable(90 DPD Contracts)

### Product wise

In [18]:
master_table_new_product=(
     master_table_new
)
Product_wise= (
        master_table_new_product
#         .filter(~f.col("Product").isNull())
        .groupBy("Product")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good=(
        Product_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad= (
        Product_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Product_wise=(
    Product_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good+total_bad))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good) - (f.col("bad") / total_bad)) *
        f.ln((f.col("good") / total_good) / (f.col("bad") / total_bad))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)

# Iv_value_Product_wise= 0.029287+0.058048
Iv_value_Product_wise=0.031090+0.048898

In [19]:
master_table_new_product.select("Product").distinct().show()

+-------------+
|      Product|
+-------------+
|Registered 3W|
| Cash in Hand|
|         #N/A|
+-------------+



In [20]:
Product_wise.toPandas()

,Product,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,#N/A,13,4,0.002215,0.002065,0.002899,3.250000,0.000283,0.764706
1,Registered 3W,3381,930,0.561621,0.537008,0.673913,3.635484,0.031090,0.784273
2,Cash in Hand,2902,446,0.436165,0.460928,0.323188,6.506726,0.048898,0.866786


In [56]:
Product_wise.toPandas().to_csv("../output/final_output/product_wise_active3w.csv")

In [22]:
Iv_value_Product_wise

0.079988

#### CUST_PERMRES

In [25]:
master_table_new_custer_pre=(
     master_table_new
)
CUST_PERMRES_wise= (
       master_table_new_custer_pre
#     .filter(~f.col("CUST_PERMRES").isNull())
    .groupBy("CUST_PERMRES")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_CUST_PERMRES_wise=(
       CUST_PERMRES_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_CUST_PERMRES_wise= (
        CUST_PERMRES_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

CUST_PERMRES_wise=(
    CUST_PERMRES_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_CUST_PERMRES_wise+total_bad_CUST_PERMRES_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_CUST_PERMRES_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_CUST_PERMRES_wise   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_CUST_PERMRES_wise) - (f.col("bad") / total_bad_CUST_PERMRES_wise)) *
        f.ln((f.col("good") / total_good_CUST_PERMRES_wise) / (f.col("bad") / total_bad_CUST_PERMRES_wise))
    )
)
     .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IV_value_CUST_PERMRES= 0.001037 + 0.091448 + 0.002238 + 0.014116
IV_value_CUST_PERMRES=0.035918+0.004745	+0.000905+0.013719

In [26]:
CUST_PERMRES_wise.toPandas().head()

,CUST_PERMRES,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,Rent,492,193,0.089239,0.078145,0.139855,2.549223,0.035918,0.718248
1,#N/A,13,4,0.002215,0.002065,0.002899,3.250000,0.000283,0.764706
2,With G S Certificate,403,114,0.067353,0.064009,0.082609,3.535088,0.004745,0.779497
3,With Parents/spouse,1057,249,0.170141,0.167884,0.180435,4.244980,0.000905,0.809342
4,With Billing Proof,4331,820,0.671053,0.687897,0.594203,5.281707,0.013719,0.840808


In [57]:
CUST_PERMRES_wise.toPandas().to_csv("../output/final_output/cust_pre_wise_active3w.csv")

In [28]:
IV_value_CUST_PERMRES

0.055287

### Guarantor PreNises

In [31]:
master_table_new_Guarantor_PreNises=(
     master_table_new
#     .filter((f.col("CUST_PERMRES")=="With G S Certificate")|
#             (f.col("CUST_PERMRES")=="Rent")|
#            (f.col("CUST_PERMRES")=="No Guarantors")|
#             (f.col("CUST_PERMRES")=="With Parents/spouse")|
#             (f.col("CUST_PERMRES")=="With Billing Proof")
            
           )

Guarantor_PreNises_wise= (
       master_table_new_Guarantor_PreNises
#     .filter(~f.col("Guarantor PreNises").isNull())
    .groupBy("Guarantor PreNises")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_Guarantor_PreNises_wise=(
       Guarantor_PreNises_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_Guarantor_PreNises_wise= (
        CUST_PERMRES_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Guarantor_PreNises_wise=(
    Guarantor_PreNises_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_Guarantor_PreNises_wise+total_bad_Guarantor_PreNises_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_Guarantor_PreNises_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_Guarantor_PreNises_wise   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_Guarantor_PreNises_wise) - (f.col("bad") / total_bad_Guarantor_PreNises_wise)) *
        f.ln((f.col("good") / total_good_Guarantor_PreNises_wise) / (f.col("bad") / total_bad_Guarantor_PreNises_wise))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)

# IV_Guarantor_PreNises_wise=4.694247e-08+5.527456e-05+2.444583e-04+1.225676e-03+3.686925e-05
IV_Guarantor_PreNises_wise=0.001995+0.004092+0.000434+0.000924+0.002485	

In [32]:
Guarantor_PreNises_wise.toPandas()

,Guarantor PreNises,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,#N/A,13,4,0.002215,0.002065,0.002899,3.250000,0.000283,0.764706
1,With G S Certificate,122,36,0.020584,0.019377,0.026087,3.388889,0.001995,0.772152
2,Rent,1039,265,0.169880,0.165025,0.192029,3.920755,0.004092,0.796779
3,With Parents/spouse,952,220,0.152684,0.151207,0.159420,4.327273,0.000434,0.812287
4,With Billing Proof,2781,582,0.438119,0.441709,0.421739,4.778351,0.000924,0.826940
5,No Guarantors,1389,273,0.216519,0.220616,0.197826,5.087912,0.002485,0.835740


In [58]:
Guarantor_PreNises_wise.toPandas().to_csv("../output/final_output/guarantor_pre_wise_active3w.csv")

In [34]:
IV_Guarantor_PreNises_wise

0.009930000000000001

### Exposure Category(Score)

In [39]:
master_table_exp=(
    master_table_new.orderBy("Exposure")
#     .filter(~f.col("Exposure").isNull())
#     .withColumn("Exposure_cat",
#                 f.when(f.col("Exposure").between(0,10),"0-10")
#                 .when(f.col("Exposure").between(11,20),"11-20")
#                 .when(f.col("Exposure").between(21,30),"21-30")
#                 .when(f.col("Exposure").between(31,40),"31-40")
#                 .when(f.col("Exposure").between(41,50),"41-50")
#                 .when(f.col("Exposure").between(51,60),"51-60")
#                 .when(f.col("Exposure").between(61,70),"61-70")
#                 .when(f.col("Exposure").between(71,80),"71-80")
#                 .when(f.col("Exposure")>80,"80>")
#                )
    .withColumn("Exposure_cat",
                f.when(f.col("Exposure").between(0,20),"0-20")
                .when(f.col("Exposure").between(21,40),"21-40")
                .when(f.col("Exposure").between(41,60),"41-60")
                .when(f.col("Exposure").between(61,80),"61-80")
                .when(f.col("Exposure")>80,"80>")
               )
)

master_table_new1=master_table_exp.groupBy("Exposure_cat").agg(f.count("FAC NO"))

Exposure_Category_Score_wise= (
        master_table_exp.groupBy("Exposure_cat")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_Exposure_Category_Score_wise=(
      Exposure_Category_Score_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_Exposure_Category_Score_wise= (
       Exposure_Category_Score_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Exposure_Category_Score_wise=(
    Exposure_Category_Score_wise
    .withColumnRenamed("Exposure Category(Score)","Exp")
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_Exposure_Category_Score_wise+total_bad_Exposure_Category_Score_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_Exposure_Category_Score_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_Exposure_Category_Score_wise  
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_Exposure_Category_Score_wise) - (f.col("bad") / total_bad_Exposure_Category_Score_wise)) *
        f.ln((f.col("good") / total_good_Exposure_Category_Score_wise) / (f.col("bad") / total_bad_Exposure_Category_Score_wise))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IVP_Exp_value=(
# 0.050055+
# 0.030004+
# 0.055117+
# 0.001172+
# 0.053804+
# 0.039878+
# 0.034502+
# 0.007726+
# 0.003169
# )
# IVP_Exp_value=0.063524+0.018908+0.005051+0.048916+0.029908
IVP_EXP_value=0.065191+0.005787+0.002526+0.025404+0.058189

In [40]:
master_table_new1.show()

+------------+-------------+
|Exposure_cat|count(FAC NO)|
+------------+-------------+
|       21-40|         1114|
|        NULL|           17|
|        0-20|          924|
|       61-80|         2187|
|         80>|         1163|
|       41-60|         2271|
+------------+-------------+



In [41]:
master_table_new1.toPandas().head()

,Exposure_cat,count(FAC NO)
0,21-40,1114
1,None,17
2,0-20,924
3,61-80,2187
4,80>,1163


In [42]:
Exposure_Category_Score_wise.toPandas().head(10)

,Exposure_cat,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,80>,831,332,0.151511,0.131989,0.240580,2.503012,0.065191,0.714531
1,None,13,4,0.002215,0.002065,0.002899,3.250000,0.000283,0.764706
2,0-20,727,197,0.120375,0.115470,0.142754,3.690355,0.005787,0.786797
3,61-80,1763,424,0.284914,0.280019,0.307246,4.158019,0.002526,0.806127
4,41-60,1956,315,0.295857,0.310673,0.228261,6.209524,0.025404,0.861295
5,21-40,1006,108,0.145128,0.159784,0.078261,9.314815,0.058189,0.903052


In [59]:
Exposure_Category_Score_wise.toPandas().to_csv("../output/final_output/exp_cat_wise_active3w.csv")

In [44]:
IVP_EXP_value

0.157097

### DSCR

In [48]:
master_table_new_dcsr=(
     master_table_new
#     .filter(~f.col("DSCR").isNull())
    .withColumn("DSCR_cat",
                f.when(f.col("DSCR").between(0,5),"0-5")
                .when(f.col("DSCR").between(6,11),"6-11")
                .when(f.col("DSCR").between(12,17),"12-17")
                .when(f.col("DSCR").between(18,23),"18-23")
                .when(f.col("DSCR")>23,"23>")
               )
)
master_table_new_dcsr1=master_table_new_dcsr.groupBy("DSCR_cat").agg(f.count("DSCR"))
DSCR= (
      master_table_new_dcsr.groupBy("DSCR_cat")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_DSCR=(
       DSCR
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_DSCR= (
        DSCR
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

DSCR=(
    DSCR
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_DSCR+total_bad_DSCR))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_DSCR   
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_DSCR 
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_DSCR) - (f.col("bad") / total_bad_DSCR)) *
        f.ln((f.col("good") / total_good_DSCR) / (f.col("bad") / total_bad_DSCR))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IV_value_DSCR= 0.000052+0.004582+0.003666+0.023002+0.000092
IV_value_DSCR=	0.004970+0.000523+0.001047+0.027438

In [49]:
master_table_new_dcsr1.show()

+--------+-----------+
|DSCR_cat|count(DSCR)|
+--------+-----------+
|   12-17|         81|
|    NULL|         17|
|     23>|        156|
|     0-5|       6479|
|    6-11|        893|
|   18-23|         50|
+--------+-----------+



In [50]:
DSCR.toPandas().head()

,DSCR_cat,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,None,13,4,0.002215,0.002065,0.002899,3.250000,0.000283,0.764706
1,0-5,5240,1239,0.844059,0.832274,0.897826,4.229217,0.004970,0.808767
2,18-23,43,7,0.006514,0.006830,0.005072,6.142857,0.000523,0.860000
3,12-17,70,11,0.010552,0.011118,0.007971,6.363636,0.001047,0.864198
4,6-11,791,102,0.116337,0.125635,0.073913,7.754902,0.027438,0.885778


In [60]:
DSCR.toPandas().to_csv("../output/final_output/dscr_cat_wise_active_3w.csv")

In [53]:
IV_value_DSCR

0.033978

IV (Information Value Statistic)	Description of Correlation
<0.01                               No Correlation
0.01 - 0.03                        	Marginal contribution
0.03 - 0.10	                        Adequate predictability
0.10 - 0.30	                        Strong predictor
0.30 - 0.50	                        Very Strong predictor
>=0.5	                            "Suspicious" - Too good to be true


In [54]:
print("IV_value_CUST_PERMRES: ",IV_value_CUST_PERMRES)
print("IV_Guarantor_PreNises_wise:",IV_Guarantor_PreNises_wise)
print("IV_value_DSCR:",IV_value_DSCR)
print("IVP_Exp_value:",IVP_EXP_value)
print("Iv_value_Product_wise:",Iv_value_Product_wise)

IV_value_CUST_PERMRES:  0.055287
IV_Guarantor_PreNises_wise: 0.009930000000000001
IV_value_DSCR: 0.033978
IVP_Exp_value: 0.157097
Iv_value_Product_wise: 0.079988


# Selecting Customer good or bad (On Total Collectiion Ratio)